In [1]:
import pickle
import re
import pandas as pd

In [2]:
scrape_path_0="../scrape_file/scrapes_atheism.pickle"
scrape_path_1="../scrape_file/scrapes_christianity.pickle"

try:
    with open(scrape_path_0,'rb') as handle:
        scrape0=pickle.load(handle)
    with open(scrape_path_1,'rb') as handle:
        scrape1=pickle.load(handle)
except FileNotFoundError as e:
    e.strerror = "Pls run 01_scrape_reddit first to pull the data."
    raise e

df0=pd.DataFrame(scrape0,columns=["post"])
df0['label']=0

df1=pd.DataFrame(scrape1,columns=["post"])
df1['label']=1

df=pd.concat([df0,df1])
df.reset_index(inplace=True,drop=True)

Let's take a look at the combined DF.

In [3]:
df

,post,label
0,It’s actually just sad that I see these pastor...,0
1,"TLDR: I love my Christian mother, but she has ...",0
2,I literally posted here like an hour or so ago...,0
3,My wife and I are in the middle of loading the...,0
4,This is the nail in the coffin for the idea of...,0
...,...,...
10076,"Jesus lived a perfect life, and died in your p...",1
10077,"Why don't nuns wear the blue coat, red dress a...",1
10078,I've had a lack of appetite for sometime and n...,1
10079,Title says it all. Wondering if anyone can giv...,1


### And the split of 1/0 labels?

In [4]:
df['label'].value_counts(normalize=True)

1    0.501339
0    0.498661
Name: label, dtype: float64

Quite an equal representation (50%) of labels are 1 and 0.  
No need to do any balancing here.

### Let's also look at how many words each post contains.

In [5]:
df['len']=df['post'].apply(lambda x: len(re.split(r'\s',x)))
df[df['label']==1]['len'].describe()
pd.DataFrame([df[df['label']==0]['len'].describe().rename("Label 0"),df[df['label']==1]['len'].describe().rename("Label 1")])

,count,mean,std,min,25%,50%,75%,max
Label 0,5027.0,173.835886,237.135363,20.0,62.0,115.0,204.0,7799.0
Label 1,5054.0,183.649189,271.955133,20.0,54.0,105.0,204.0,4996.0


No real difference between the lengths of post between the 2 categories.

Let's move on to pump the entire dataframe into a preliminary model next.